In [2]:
SELECT 
    YEAR(order_date) AS Year,
    MONTH(order_date) AS Month,
    round(SUM(total_sales),2) AS TotalSales
FROM 
    [dbo].[Sales]
WHERE 
    category = 'Furniture'
GROUP BY 
    YEAR(order_date), MONTH(order_date)
ORDER BY 
    YEAR(order_date), MONTH(order_date);

(48 rows affected)

Total execution time: 00:00:00.033

Year,Month,TotalSales
2014,1,6242.52
2014,2,1839.66
2014,3,14573.96
2014,4,7944.84
2014,5,6912.79
2014,6,13206.13
2014,7,10821.05
2014,8,7320.35
2014,9,23816.48
2014,10,12304.25


In [3]:
WITH SalesByMonth AS (
    SELECT 
        YEAR(order_date) AS Year,
        MONTH(order_date) AS Month,
        DATEFROMPARTS(YEAR(order_date), MONTH(order_date), 1) AS SalesMonth,
        SUM(total_sales) AS TotalSales
    FROM 
        [dbo].[Sales]
    WHERE 
        category = 'Furniture'
    GROUP BY 
        YEAR(order_date), MONTH(order_date)
)
SELECT 
    SalesMonth,
    SUM(TotalSales) OVER (
        ORDER BY SalesMonth 
        ROWS BETWEEN 11 PRECEDING AND CURRENT ROW
    ) AS Rolling12MonthSales
FROM 
    SalesByMonth
ORDER BY 
    SalesMonth;

(48 rows affected)

Total execution time: 00:00:00.012

SalesMonth,Rolling12MonthSales
2014-01-01,6242.525
2014-02-01,8082.183
2014-03-01,22656.139
2014-04-01,30600.976000000002
2014-05-01,37513.763000000006
2014-06-01,50719.888600000006
2014-07-01,61540.939600000005
2014-08-01,68861.2861
2014-09-01,92677.7669
2014-10-01,104982.0139


In [5]:
WITH MonthlySales AS (
    SELECT 
        CONCAT(YEAR(order_date), '-', RIGHT('00' + CAST(MONTH(order_date) AS VARCHAR(2)), 2)) AS YearMonth,
        sub_category,
        SUM(total_sales) AS TotalSales
    FROM 
        [dbo].[Sales]
    WHERE 
        category = 'Furniture'
    GROUP BY 
        YEAR(order_date), MONTH(order_date), sub_category
),
SalesWithPreviousMonth AS (
    SELECT 
        ms.YearMonth,
        ms.sub_category,
        ms.TotalSales,
        LAG(ms.TotalSales, 1) OVER (PARTITION BY ms.sub_category ORDER BY ms.YearMonth) AS PreviousMonthSales
    FROM 
        MonthlySales ms
)
SELECT 
    YearMonth,
    sub_category,
    TotalSales,
    PreviousMonthSales,
    CASE 
        WHEN PreviousMonthSales IS NOT NULL AND PreviousMonthSales > 0 THEN 
            ((TotalSales - PreviousMonthSales) / PreviousMonthSales) * 100 
        ELSE NULL 
    END AS MonthlyGrowthRate
FROM 
    SalesWithPreviousMonth
ORDER BY 
    sub_category, YearMonth;

(190 rows affected)

Total execution time: 00:00:00.032

YearMonth,sub_category,TotalSales,PreviousMonthSales,MonthlyGrowthRate
2014-01,Bookcases,1010.0550000000001,NULL,NULL
2014-03,Bookcases,1706.446,1010.0550000000001,68.94584948344395
2014-04,Bookcases,308.499,1706.446,-81.9215492315608
2014-05,Bookcases,640.631,308.499,107.66064071520489
2014-06,Bookcases,759.5735999999999,640.631,18.56647586520165
2014-07,Bookcases,1487.673,759.5735999999999,95.85633307950673
2014-08,Bookcases,794.2760000000001,1487.673,-46.6095035669801
2014-09,Bookcases,2394.4698,794.2760000000001,201.46571217058047
2014-10,Bookcases,616.998,2394.4698,-74.23237494997849
2014-11,Bookcases,7263.713700000001,616.998,1077.2669765542191


## Project 9: Profitability Analysis by Region

### Objective: Analyze profitability across different regions to optimize regional sales strategies.

In [5]:
-- 1. Calculate the total profit and profit margin for each region.

SELECT 
    region,
    SUM(profit) AS total_profit,
    (SUM(profit) / SUM(CAST(total_sales AS FLOAT))) * 100 AS profit_margin
FROM 
    Sales
GROUP BY 
    region;


(4 rows affected)

Total execution time: 00:00:00.010

region,total_profit,profit_margin
East,3046.1657999999993,1.4624553228853567
South,6771.206099999996,5.772618983517321
West,11504.95029999999,4.55438238807615
Central,-2871.0494000000003,-1.7528077613759032


In [9]:
-- 2. Rank regions by profitability and total sales volume

WITH RegionProfitSales AS (
    SELECT 
        region,
        SUM(profit) AS total_profit,
        SUM(CAST(total_sales AS FLOAT)) AS total_sales
    FROM 
        Sales
    GROUP BY 
        region
)
SELECT 
    region,
    total_profit,
    RANK() OVER (ORDER BY total_profit DESC) AS profitability_rank,
    total_sales,
    RANK() OVER (ORDER BY total_sales DESC) AS sales_volume_rank
FROM 
    RegionProfitSales
ORDER BY sales_volume_rank;

(4 rows affected)

Total execution time: 00:00:00.018

region,total_profit,profitability_rank,total_sales,sales_volume_rank
West,11504.95029999999,1,252612.74350000022,1
East,3046.1657999999993,3,208291.20400000017,2
Central,-2871.0494000000003,4,163797.16379999995,3
South,6771.206099999996,2,117298.68400000002,4


In [11]:
--  3. Identify regions with declining sales and provide insights on potential causes.
-- Identifying Declining Sales: We can compare the total sales year-over-year (from 2014 to 2017) for each region. 
-- If the sales decrease across consecutive years, that region can be flagged as having declining sales.
WITH YearlySales AS (
    SELECT 
        region,
        YEAR(order_date) AS order_year,
        SUM(CAST(total_sales AS FLOAT)) AS total_sales
    FROM 
        Sales
    GROUP BY 
        region, YEAR(order_date)
)
SELECT 
    region,
    order_year,
    total_sales,
    LAG(total_sales, 1) OVER (PARTITION BY region ORDER BY order_year) AS previous_year_sales,
    (total_sales - LAG(total_sales, 1) OVER (PARTITION BY region ORDER BY order_year)) AS sales_change
FROM 
    YearlySales
ORDER BY 
    region, order_year;


(16 rows affected)

Total execution time: 00:00:00.019

region,order_year,total_sales,previous_year_sales,sales_change
Central,2014,32909.663600000014,NULL,NULL
Central,2015,35592.04699999999,32909.663600000014,2682.383399999977
Central,2016,50773.182,35592.04699999999,15181.13500000001
Central,2017,44522.27119999998,50773.182,-6250.91080000002
East,2014,47232.73900000003,NULL,NULL
East,2015,53817.43199999998,47232.73900000003,6584.692999999948
East,2016,46387.171999999984,53817.43199999998,-7430.259999999995
East,2017,60853.86100000003,46387.171999999984,14466.68900000005
South,2014,26968.002499999988,NULL,NULL
South,2015,24103.814499999993,26968.002499999988,-2864.1879999999946


In [12]:
WITH YearlySalesDetails AS (
    SELECT 
        region,
        YEAR(order_date) AS order_year,
        ship_mode,
        SUM(CAST(total_sales AS FLOAT)) AS total_sales
    FROM 
        Sales
    GROUP BY 
        region, YEAR(order_date), ship_mode
)
SELECT 
    region,
    order_year,
    ship_mode,
    total_sales,
    LAG(total_sales, 1) OVER (PARTITION BY region, ship_mode ORDER BY order_year) AS previous_year_sales,
    (total_sales - LAG(total_sales, 1) OVER (PARTITION BY region, ship_mode ORDER BY order_year)) AS sales_change
FROM 
    YearlySalesDetails
ORDER BY 
    region, order_year, ship_mode;


(64 rows affected)

Total execution time: 00:00:00.036

region,order_year,ship_mode,total_sales,previous_year_sales,sales_change
Central,2014,First Class,5492.114,NULL,NULL
Central,2014,Same Day,25.16,NULL,NULL
Central,2014,Second Class,6984.8,NULL,NULL
Central,2014,Standard Class,20407.589600000003,NULL,NULL
Central,2015,First Class,3291.055,5492.114,-2201.0589999999997
Central,2015,Same Day,3858.2040000000006,25.16,3833.044000000001
Central,2015,Second Class,5048.044400000001,6984.8,-1936.7555999999995
Central,2015,Standard Class,23394.743599999987,20407.589600000003,2987.153999999984
Central,2016,First Class,4057.5588000000002,3291.055,766.5038000000004
Central,2016,Same Day,2514.702,3858.2040000000006,-1343.5020000000004


In [13]:
--  4. Analyze which product categories and sub-categories are most profitable 
-- in each region.
WITH CategoryProfit AS (
    SELECT
        region,
        category,
        sub_category,
        SUM(profit) AS total_profit
    FROM
        Sales
    GROUP BY
        region, category, sub_category
)
SELECT
    region,
    category,
    sub_category,
    total_profit,
    RANK() OVER (PARTITION BY region ORDER BY total_profit DESC) AS profitability_rank
FROM
    CategoryProfit
ORDER BY
    region, profitability_rank;


(16 rows affected)

Total execution time: 00:00:00.028

region,category,sub_category,total_profit,profitability_rank
Central,Furniture,Chairs,6592.722100000002,1
Central,Furniture,Bookcases,-1997.9043000000004,2
Central,Furniture,Tables,-3559.6503999999995,3
Central,Furniture,Furnishings,-3906.216800000001,4
East,Furniture,Chairs,9357.7706,1
East,Furniture,Furnishings,5881.407099999999,2
East,Furniture,Bookcases,-1167.6317999999999,3
East,Furniture,Tables,-11025.380100000002,4
South,Furniture,Chairs,6612.0893,1
South,Furniture,Furnishings,3442.682899999999,2


In [19]:
--  5. Create a report that recommends regions for increased marketing 
-- investment based on profitability and sales growth potential
 WITH ProfitAndSalesGrowth AS (
    -- Calculate total profit and sales for each region and year
    SELECT 
        region,
        YEAR(order_date) AS order_year,
        SUM(CAST(total_sales AS FLOAT)) AS total_sales,
        SUM(profit) AS total_profit
    FROM 
        Sales
    GROUP BY 
        region, YEAR(order_date)
),
SalesGrowth AS (
    -- Calculate year-over-year sales growth for each region
    SELECT 
        region,
        order_year,
        total_sales,
        LAG(total_sales, 1) OVER (PARTITION BY region ORDER BY order_year) AS previous_year_sales,
        total_profit,
        (total_sales - LAG(total_sales, 1) OVER (PARTITION BY region ORDER BY order_year)) AS sales_change
    FROM 
        ProfitAndSalesGrowth
),
RegionAnalysis AS (
    -- Summarize sales growth and profitability for each region
    SELECT 
        region,
        SUM(total_profit) AS total_profit,
        SUM(CASE WHEN sales_change > 0 THEN 1 ELSE 0 END) AS positive_sales_years,
        COUNT(*) - 1 AS total_years -- Subtract 1 to ignore the first year, as it has no previous year comparison
    FROM 
        SalesGrowth
    GROUP BY 
        region
)
-- Generate the final recommendation report
SELECT 
    region,
    total_profit,
    (positive_sales_years * 1.0 / total_years) * 100 AS sales_growth_percentage,
    CASE 
        WHEN total_profit > (SELECT AVG(total_profit) FROM RegionAnalysis) 
             AND (positive_sales_years * 1.0 / total_years) > 0.5 
        THEN 'Recommended for increased marketing investment'
        ELSE 'Not recommended'
    END AS marketing_recommendation
FROM 
    RegionAnalysis
ORDER BY 
    total_profit DESC, sales_growth_percentage DESC;


(4 rows affected)

Total execution time: 00:00:00.040

region,total_profit,sales_growth_percentage,marketing_recommendation
West,11504.9503,66.666666666600,Recommended for increased marketing investment
South,6771.2061,66.666666666600,Recommended for increased marketing investment
East,3046.1658000000007,66.666666666600,Not recommended
Central,-2871.049399999999,66.666666666600,Not recommended


## Project 10: Advanced Sales Growth Analytics

### Objective: Dive deeper into sales growth metrics and analyze factors influencing growth.

In [4]:
-- 1. Calculate the year-over-year (YoY) and quarter-over-quarter (QoQ) sales growth for the entire
-- business.
SELECT 
    YEAR(order_date) AS order_year, 
    DATEPART(QUARTER, order_date) AS order_quarter, 
    SUM(CAST(total_sales AS DECIMAL(18, 2))) AS total_sales
FROM sales
GROUP BY 
    YEAR(order_date), 
    DATEPART(QUARTER, order_date)
ORDER BY 
    YEAR(order_date), 
    DATEPART(QUARTER, order_date);


(16 rows affected)

Total execution time: 00:00:00.021

order_year,order_quarter,total_sales
2014,1,22656.17
2014,2,28063.72
2014,3,41957.89
2014,4,64515.06
2015,1,27374.09
2015,2,27564.85
2015,3,49586.03
2015,4,65993.27
2016,1,24349.41
2016,2,41402.49


In [5]:
-- Step 2: Calculate Year-Over-Year (YoY) Growth

WITH YearlySales AS (
    SELECT 
        YEAR(order_date) AS order_year, 
        SUM(CAST(total_sales AS DECIMAL(18, 2))) AS total_sales
    FROM sales
    GROUP BY YEAR(order_date)
)
SELECT 
    order_year,
    total_sales,
    LAG(total_sales) OVER (ORDER BY order_year) AS previous_year_sales,
    CASE 
        WHEN LAG(total_sales) OVER (ORDER BY order_year) IS NOT NULL THEN 
            (total_sales - LAG(total_sales) OVER (ORDER BY order_year)) / LAG(total_sales) OVER (ORDER BY order_year) * 100
        ELSE 0
    END AS YoY_growth
FROM YearlySales;


(4 rows affected)

Total execution time: 00:00:00.024

order_year,total_sales,previous_year_sales,YoY_growth
2014,157192.84,NULL,0.000000
2015,170518.24,157192.84,8.477100
2016,198901.48,170518.24,16.645200
2017,215387.19,198901.48,8.288300


In [6]:
-- Step 3: Calculate Quarter-Over-Quarter (QoQ) Growth
WITH QuarterlySales AS (
    SELECT 
        YEAR(order_date) AS order_year, 
        DATEPART(QUARTER, order_date) AS order_quarter, 
        SUM(CAST(total_sales AS DECIMAL(18, 2))) AS total_sales
    FROM sales
    GROUP BY YEAR(order_date), DATEPART(QUARTER, order_date)
)
SELECT 
    order_year,
    order_quarter,
    total_sales,
    LAG(total_sales) OVER (ORDER BY order_year, order_quarter) AS previous_quarter_sales,
    CASE 
        WHEN LAG(total_sales) OVER (ORDER BY order_year, order_quarter) IS NOT NULL THEN 
            (total_sales - LAG(total_sales) OVER (ORDER BY order_year, order_quarter)) / LAG(total_sales) OVER (ORDER BY order_year, order_quarter) * 100
        ELSE 0
    END AS QoQ_growth
FROM QuarterlySales;


(16 rows affected)

Total execution time: 00:00:00.025

order_year,order_quarter,total_sales,previous_quarter_sales,QoQ_growth
2014,1,22656.17,NULL,0.000000
2014,2,28063.72,22656.17,23.867800
2014,3,41957.89,28063.72,49.509300
2014,4,64515.06,41957.89,53.761400
2015,1,27374.09,64515.06,-57.569400
2015,2,27564.85,27374.09,0.696800
2015,3,49586.03,27564.85,79.888600
2015,4,65993.27,49586.03,33.088400
2016,1,24349.41,65993.27,-63.103100
2016,2,41402.49,24349.41,70.034800


In [7]:
-- 2. Identify key drivers of sales growth, such as market segment or region performance.
-- ** 1. Sales Growth by Market Segment
WITH YearlySegmentSales AS (
    SELECT 
        YEAR(order_date) AS order_year, 
        market_segment, 
        SUM(CAST(total_sales AS DECIMAL(18, 2))) AS total_sales
    FROM sales
    GROUP BY YEAR(order_date), market_segment
)
SELECT 
    order_year,
    market_segment,
    total_sales,
    LAG(total_sales) OVER (PARTITION BY market_segment ORDER BY order_year) AS previous_year_sales,
    CASE 
        WHEN LAG(total_sales) OVER (PARTITION BY market_segment ORDER BY order_year) IS NOT NULL THEN 
            (total_sales - LAG(total_sales) OVER (PARTITION BY market_segment ORDER BY order_year)) / LAG(total_sales) OVER (PARTITION BY market_segment ORDER BY order_year) * 100
        ELSE 0
    END AS YoY_growth
FROM YearlySegmentSales;


(12 rows affected)

Total execution time: 00:00:00.040

order_year,market_segment,total_sales,previous_year_sales,YoY_growth
2014,Consumer,86060.98,NULL,0.000000
2015,Consumer,94409.92,86060.98,9.701100
2016,Consumer,100360.92,94409.92,6.303300
2017,Consumer,110217.51,100360.92,9.821100
2014,Corporate,45100.28,NULL,0.000000
2015,Corporate,45961.10,45100.28,1.908600
2016,Corporate,77261.34,45961.10,68.101500
2017,Corporate,60697.04,77261.34,-21.439300
2014,Home Office,26031.58,NULL,0.000000
2015,Home Office,30147.22,26031.58,15.810100


In [8]:
-- 2. Sales Growth by Region
WITH YearlyRegionSales AS (
    SELECT 
        YEAR(order_date) AS order_year, 
        region, 
        SUM(CAST(total_sales AS DECIMAL(18, 2))) AS total_sales
    FROM sales
    GROUP BY YEAR(order_date), region
)
SELECT 
    order_year,
    region,
    total_sales,
    LAG(total_sales) OVER (PARTITION BY region ORDER BY order_year) AS previous_year_sales,
    CASE 
        WHEN LAG(total_sales) OVER (PARTITION BY region ORDER BY order_year) IS NOT NULL THEN 
            (total_sales - LAG(total_sales) OVER (PARTITION BY region ORDER BY order_year)) / LAG(total_sales) OVER (PARTITION BY region ORDER BY order_year) * 100
        ELSE 0
    END AS YoY_growth
FROM YearlyRegionSales;


(16 rows affected)

Total execution time: 00:00:00.093

order_year,region,total_sales,previous_year_sales,YoY_growth
2014,Central,32909.66,NULL,0.000000
2015,Central,35592.06,32909.66,8.150700
2016,Central,50773.16,35592.06,42.653000
2017,Central,44522.26,50773.16,-12.311400
2014,East,47232.72,NULL,0.000000
2015,East,53817.38,47232.72,13.940800
2016,East,46387.20,53817.38,-13.806200
2017,East,60853.83,46387.20,31.186600
2014,South,26968.01,NULL,0.000000
2015,South,24103.83,26968.01,-10.620600


In [11]:
WITH RegionSales AS (
    SELECT 
        region,
        YEAR(order_date) AS order_year, 
        SUM(CAST(total_sales AS DECIMAL(18, 2))) AS total_sales
    FROM sales
    GROUP BY region, YEAR(order_date)
),
FirstLastSales AS (
    SELECT 
        region, 
        order_year, 
        total_sales,
        ROW_NUMBER() OVER (PARTITION BY region ORDER BY order_year ASC) AS rn_asc,
        ROW_NUMBER() OVER (PARTITION BY region ORDER BY order_year DESC) AS rn_desc
    FROM RegionSales
)
-- Self join to get the first and last year sales for each region
SELECT 
    f.region,
    f.order_year AS first_year,
    l.order_year AS last_year,
    f.total_sales AS sales_first_year,
    l.total_sales AS sales_last_year,
    CASE 
        WHEN f.total_sales > 0 THEN 
            ((l.total_sales - f.total_sales) / f.total_sales) * 100
        ELSE NULL 
    END AS total_growth_percent
FROM FirstLastSales f
JOIN FirstLastSales l
    ON f.region = l.region
    AND f.rn_asc = 1 -- first year sales
    AND l.rn_desc = 1 -- last year sales
ORDER BY total_growth_percent DESC;


(4 rows affected)

Total execution time: 00:00:00.043

region,first_year,last_year,sales_first_year,sales_last_year,total_growth_percent
West,2014,2017,50082.45,71705.70,43.175300
South,2014,2017,26968.01,38305.40,42.040100
Central,2014,2017,32909.66,44522.26,35.286200
East,2014,2017,47232.72,60853.83,28.838200


In [12]:
-- 3. Analyze which customer segments are contributing the most to sales growth.

WITH SegmentSales AS (
    SELECT 
        market_segment,
        YEAR(order_date) AS order_year,
        SUM(CAST(total_sales AS DECIMAL(18, 2))) AS total_sales
    FROM sales
    GROUP BY market_segment, YEAR(order_date)
),
SegmentGrowth AS (
    SELECT 
        market_segment, 
        order_year,
        total_sales,
        LAG(total_sales) OVER (PARTITION BY market_segment ORDER BY order_year) AS previous_year_sales,
        CASE 
            WHEN LAG(total_sales) OVER (PARTITION BY market_segment ORDER BY order_year) IS NOT NULL THEN
                ((total_sales - LAG(total_sales) OVER (PARTITION BY market_segment ORDER BY order_year)) / 
                LAG(total_sales) OVER (PARTITION BY market_segment ORDER BY order_year)) * 100
            ELSE 0
        END AS YoY_growth
    FROM SegmentSales
)
SELECT 
    market_segment,
    SUM(total_sales) AS total_sales_over_period,
    AVG(YoY_growth) AS avg_YoY_growth
FROM SegmentGrowth
GROUP BY market_segment
ORDER BY avg_YoY_growth DESC;


(3 rows affected)

Total execution time: 00:00:00.044

market_segment,total_sales_over_period,avg_YoY_growth
Home Office,121930.66,23.847525
Corporate,229019.76,12.142700
Consumer,391049.33,6.456375


In [13]:
--  4. Create a heatmap to visualize growth by product category and sub-category

WITH CategorySubcategorySales AS (
    SELECT 
        category,
        sub_category,
        YEAR(order_date) AS order_year,
        SUM(CAST(total_sales AS DECIMAL(18, 2))) AS total_sales
    FROM sales
    GROUP BY category, sub_category, YEAR(order_date)
),
CategorySubcategoryGrowth AS (
    SELECT 
        category,
        sub_category,
        order_year,
        total_sales,
        LAG(total_sales) OVER (PARTITION BY category, sub_category ORDER BY order_year) AS previous_year_sales,
        CASE 
            WHEN LAG(total_sales) OVER (PARTITION BY category, sub_category ORDER BY order_year) IS NOT NULL THEN
                ((total_sales - LAG(total_sales) OVER (PARTITION BY category, sub_category ORDER BY order_year)) / 
                LAG(total_sales) OVER (PARTITION BY category, sub_category ORDER BY order_year)) * 100
            ELSE 0
        END AS YoY_growth
    FROM CategorySubcategorySales
)
SELECT 
    category,
    sub_category,
    order_year,
    YoY_growth
FROM CategorySubcategoryGrowth
ORDER BY category, sub_category, order_year;


(16 rows affected)

Total execution time: 00:00:00.041

category,sub_category,order_year,YoY_growth
Furniture,Bookcases,2014,0.000000
Furniture,Bookcases,2015,92.365000
Furniture,Bookcases,2016,-31.829200
Furniture,Bookcases,2017,14.267300
Furniture,Chairs,2014,0.000000
Furniture,Chairs,2015,-7.129500
Furniture,Chairs,2016,16.985000
Furniture,Chairs,2017,13.865400
Furniture,Furnishings,2014,0.000000
Furniture,Furnishings,2015,52.533800


In [14]:
-- Develop a predictive model for future growth based on past performance metrics and market
 -- conditions

 WITH HistoricalData AS (
    SELECT 
        YEAR(order_date) AS order_year,
        DATEPART(QUARTER, order_date) AS order_quarter,
        market_segment,
        region,
        category,
        sub_category,
        SUM(CAST(total_sales AS DECIMAL(18, 2))) AS total_sales
    FROM sales
    GROUP BY YEAR(order_date), DATEPART(QUARTER, order_date), market_segment, region, category, sub_category
),
LaggedData AS (
    SELECT 
        *,
        LAG(total_sales) OVER (PARTITION BY market_segment, region, category, sub_category ORDER BY order_year, order_quarter) AS previous_quarter_sales,
        LAG(total_sales, 4) OVER (PARTITION BY market_segment, region, category, sub_category ORDER BY order_year, order_quarter) AS previous_year_sales,
        (total_sales - LAG(total_sales, 4) OVER (PARTITION BY market_segment, region, category, sub_category ORDER BY order_year, order_quarter)) / 
        LAG(total_sales, 4) OVER (PARTITION BY market_segment, region, category, sub_category ORDER BY order_year, order_quarter) * 100 AS YoY_growth,
        (total_sales - LAG(total_sales) OVER (PARTITION BY market_segment, region, category, sub_category ORDER BY order_year, order_quarter)) / 
        LAG(total_sales) OVER (PARTITION BY market_segment, region, category, sub_category ORDER BY order_year, order_quarter) * 100 AS QoQ_growth
    FROM HistoricalData
)
SELECT 
    order_year,
    order_quarter,
    market_segment,
    region,
    category,
    sub_category,
    total_sales,
    previous_quarter_sales,
    previous_year_sales,
    YoY_growth,
    QoQ_growth
FROM LaggedData;


(590 rows affected)

Total execution time: 00:00:00.069

order_year,order_quarter,market_segment,region,category,sub_category,total_sales,previous_quarter_sales,previous_year_sales,YoY_growth,QoQ_growth
2014,2,Consumer,Central,Furniture,Bookcases,193.07,NULL,NULL,NULL,NULL
2014,3,Consumer,Central,Furniture,Bookcases,493.43,193.07,NULL,NULL,155.570500
2014,4,Consumer,Central,Furniture,Bookcases,424.12,493.43,NULL,NULL,-14.046500
2015,1,Consumer,Central,Furniture,Bookcases,1865.34,424.12,NULL,NULL,339.814200
2015,3,Consumer,Central,Furniture,Bookcases,1298.37,1865.34,193.07,572.486600,-30.394900
2015,4,Consumer,Central,Furniture,Bookcases,973.06,1298.37,493.43,97.203200,-25.055200
2016,1,Consumer,Central,Furniture,Bookcases,241.33,973.06,424.12,-43.098600,-75.198800
2016,3,Consumer,Central,Furniture,Bookcases,2751.38,241.33,1865.34,47.500100,1040.090300
2016,4,Consumer,Central,Furniture,Bookcases,1704.48,2751.38,1298.37,31.278400,-38.049900
2017,1,Consumer,Central,Furniture,Bookcases,1023.33,1704.48,973.06,5.166100,-39.962300
